In [1]:
from pynq import Overlay
from pynq.overlays.base import BaseOverlay
import time
import numpy as np
from pynq import allocate

In [2]:
ol = BaseOverlay('bitstreams/mission_4adc_re3.bit')

In [3]:
#ol?
burst_gate = ol.loop_hier.burst_rec.burst_gate_ctrl.register_map.GPIO_DATA
packager = ol.loop_hier.burst_rec.packager_ctrl.register_map.GPIO_DATA

trig1 = ol.trigger_hier.double_trig_hier.trig1_ctrl.register_map.GPIO_DATA #Master Trigger
trig2 = ol.trigger_hier.double_trig_hier.trig2_ctrl.register_map.GPIO_DATA #DAC Trigger
trig3 = ol.trigger_hier.double_trig_hier.trig3_ctrl.register_map.GPIO_DATA #ADC Trigger
trig4 = ol.trigger_hier.double_trig_hier.trig4_ctrl.register_map.GPIO_DATA #Trigger3
trig5 = ol.trigger_hier.double_trig_hier.trig5_ctrl.register_map.GPIO_DATA #Trigger4

pps = ol.trigger_hier.double_trig_hier.pps_ctrl.register_map.GPIO_DATA #pps


dac_gate = ol.loop_hier.ring_transmit.dac_gate_ctrl.register_map.GPIO_DATA
ringer = ol.loop_hier.ring_transmit.ringer_ctrl.register_map.GPIO_DATA

pmod = ol.gpio_hier.pmod_ctrl.register_map.GPIO_DATA

In [4]:
ol.cmac.internal_loopback = 1
ol.cmac.start()

In [5]:
single_burst = False

# Transmit & trigger parameters

## Modify the following cell to adjust chirp length

In [ ]:
# 3000 m. agl
chirp_length = 16.5e-6 # in seconds
rx_window_delay = 24e-6 # in seconds

# 2000 m. agl
chirp_length = 14e-6  # in seconds
rx_window_delay = 21.5e-6  # in seconds

In [7]:
def init_recv(pcksize=50, num_pcks_per_burst=250, header_size=4, wait=300, data_option = 1):
    burstsize = pcksize*num_pcks_per_burst
    burst_gate[15:1] = burstsize
    burst_gate[0] = 0
    burst_gate[16] = data_option
    packager[15:1] = pcksize
    packager[31:16] = wait

def start_burst(use_gps = 0):
    trig3[0] = 1 # enable_trigger2
    pps[0] = use_gps
    packager[0] = 1 #start transmission
    burst_gate[0] = 1 #start transmission

def end_burst(use_gps = 0):
    trig3[0] = 0 # enable_trigger2
    pps[0] = 0
    time.sleep(0.5)
    packager[0] = 0
    burst_gate[0] = 0



In [8]:
# Transmit Control
def clear_start_fifo():
    ringer[1] = 0 #Disable
    ringer[2] = 0 #Set to PS mode
    ringer[0] = 1 #Burn
    time.sleep(0.5)
    ringer[0] = 0 #Stop Burn
    

def clear_ring_fifo():
    ringer[1] = 0 #Disable
    ringer[2] = 1 #Set to PL mode
    dac_gate[0] = 0 #Stop Transmission
    dac_gate[1] = 1 #Burn
    time.sleep(0.5)
    dac_gate[1] = 0 #Stop Burn


def load_ring(burst_length = 201):
    ringer[0] = 0 #Stop Burn
    ringer[1] = 1 #Enable
    ringer[2] = 0 #Route to PS Mode
    if not single_burst:
        print("Sending data to DMA")
        ol.dma.sendchannel.transfer(dma_buf_in)
        ol.dma.sendchannel.wait()
        time.sleep(0.5)
    else:
        burst[0] = 0
        burst[15:1] = burst_length
        burst[0] = 1
        time.sleep(1)
        burst[0] = 0
        
    
def ring_mode():
    ringer[1] = 0 #Disable
    ringer[2] = 1 #Set to PL Mode
    ringer[1] = 1 #Enable
    dac_gate[0] = 1 #Enable Transmission
    
def transmit():
    #external trigs
    trig4[0] = 1
    trig5[0] = 1
    #dac trig
    trig2[0] = 0
    dac_gate[0] = 1
    trig2[0] = 1

def start_transmit():
    clear_start_fifo()
    clear_ring_fifo()
    load_ring()
    ring_mode()
    transmit()

def stop_transmit():
    #external trigs
    trig4[0] = 0
    trig5[0] = 0
    #dac trig
    trig2[0] = 0
    dac_gate[0] = 0

In [9]:
def generate_chirp(num_samples, f0, f1, sample_rate):
    """
    Generates a chirp signal.
    
    Parameters:
    num_samples (int): Number of samples in the chirp signal.
    f0 (float): Starting frequency of the chirp in Hz.
    f1 (float): Ending frequency of-len the chirp in Hz.
    sample_rate (int): Sampling rate in samples per second.
    
    Returns:
    numpy.ndarray: The generated chirp signal as a numpy array of type int16.
    """
    t = np.linspace(0, num_samples / sample_rate, num_samples)
    k = (f1 - f0) / (num_samples / sample_rate)
    chirp_signal = np.cos(2 * np.pi * (f0 * t + 0.5 * k * t**2))
    
    # Normalize to the range of int16
    chirp_signal = np.int16(chirp_signal / np.max(np.abs(chirp_signal)) * (2**14 - 1)) # CHANGE BACK TO 14
    
    return chirp_signal

# Parameters
f0 = 100e6
f1 = 180e6
sample_rate = 153.6e6 * 16# 245.6 Mhz

signal_length = int(64*50*16 * 2  * 2)

#Formula: int(sample_rate * pulse_width)
# pulse_width = 20 #in micro
# n_samples = int(sample_rate * pulse_width)
# n_samples = 49120 #20micro
# n_samples = 27016 #11 micro
# n_samples = 40524 # 16.5u s

n_samples = int(sample_rate * chirp_length) 

chirp_signal = generate_chirp(n_samples, f0, f1, sample_rate)
chirp_signal = np.concatenate([chirp_signal, np.zeros((signal_length-n_samples*4)//4, np.int16)])





In [10]:
len(chirp_signal)

51200

In [11]:
sine16_wave = chirp_signal

# Extract the lower 8 bits (least significant byte)
lower_bytes = sine16_wave & 0xFF

# Extract the upper 8 bits (most significant byte), then shift them to the right
upper_bytes = (sine16_wave >> 8) & 0xFF

# Combine lower and upper bytes into a uint8 array
# uint8_array = np.column_stack((upper_bytes, lower_bytes)).astype(np.uint8)
uint8_array = np.column_stack((lower_bytes, upper_bytes)).astype(np.uint8)


# Flatten the array to get a single array of uint8 elements
sine_wave = uint8_array.flatten()

# Assume sine_wave and zeros_array are the two arrays of equal length
zeros_array = np.zeros_like(sine_wave, dtype=np.uint8)

# Create an empty array of double the length of one of the arrays
interleaved_array = np.empty(sine_wave.size + zeros_array.size, dtype=np.uint8)

# Fill the interleaved array
m = 32
interleaved_array = np.reshape(interleaved_array, (-1,m))
sine_wave = np.reshape(sine_wave, (-1, m))
zeros_array = np.reshape(zeros_array, (-1,m))
interleaved_array[0::2] = sine_wave
interleaved_array[1::2] = zeros_array

signal = interleaved_array.flatten()

In [12]:
# signal = np.concatenate([np.zeros_like(signal[::8]), signal, np.zeros_like(signal[::64])])  #delay too small
signal = np.concatenate([np.zeros_like(signal[::4]), signal, np.zeros_like(signal[::64])]) #delay large enough
signal_length = len(signal) 

In [13]:
dma_buf_in = allocate(signal.shape[0], dtype=np.uint8)
dma_buf_in[:] = signal
single_burst = False
# dac_gate[2] = 1
# dac_gate[3] = 0
# dac_gate[31:4] = (signal_length//32 - 2) // 2
dac_gate[31:2] = (signal_length//32 - 2) // 2

In [14]:
def init_master_trig(period = 153600): #-4
    trig1[31:8] = period
    trig1[7:2] = 3
    trig1[1] = 1
    trig1[0] = 1

# One digital value increment:
#    (cycles_high) = (2^4)/153.6e6 = 104.167 ns
#    (delay) = (2^5)/153.6e6 = 208.33 ns

cycles_high = 2**4 / 153.6e6 # ns/samp
delay = 2**5 / 153.6e6 # ns/samp

# Calculate trigger delays based on system specifications
ku_buffer = 2e-6
inherent_delay = 5e-6
capella_buffer = 15e-6

delay_tx = capella_buffer - inherent_delay
delay_tx_samp = int(np.ceil(delay_tx / delay))

delay_rx = delay_tx + rx_window_delay
delay_rx_samp = int(np.ceil(delay_rx / delay))

delay_ku = capella_buffer - ku_buffer
delay_ku_samp = int(np.ceil(delay_ku / delay))

# Calculate trigger high time based on system specifications
c_len = capella_buffer * 2 + chirp_length
c_len_samp = int(np.ceil(c_len / cycles_high))

ku_len = 2 * ku_buffer + chirp_length
ku_len_samp = int(np.ceil(ku_len / cycles_high))

def init_export_trigs(
        delay1 = delay_tx_samp, high1 = 240,           # DAC - high = default value
        delay2 = delay_rx_samp, high2 = 240,           # RCV window - high = default value
        delay3 = delay_ku_samp, high3 = ku_len_samp,   # Ku
        delay4 = 0, high4 = c_len_samp):               # C & Capella - Reference delay
    
    # DAC trigger - Synced with chirp
    trig2[31:13] = delay1 #Dac Trigger, Cycles Delayed
    trig2[12:1] = high1 # Dac Trigger, Cycles High
    
    # RCV window
    trig3[31:13] = delay2 #Export Trigger3, Cycles Delayed
    trig3[12:1] = high2 # Export Trigger3, Cycles High
    
    # Ku trigger
    trig4[31:13] = delay3 #Export Trigger4, Cycles Delayed
    trig4[12:1] = high3 # Export Trigger4, Cycles High
    
    # C/Capella trigger
    trig5[31:13] = delay4 #Export Trigger5, Cycles Delayed
    trig5[12:1] = high4 # Export Trigger5, Cycles High

    
def test_exports(active = True, gps = True):
    if gps:
        pmod[3:0] = 0
        pmod[7:4] = 0
        pmod[0] = 1
        pmod[10:8] = 2
    else:
        pmod[7:0] = 0
        pmod[10:8] = 7
    pmod[14:11] = 0
    # pmod[11] = 1
    # pmod[12] = 1
    pmod[13] = 1 # Invert trigger
    # pmod[14] = 1
    


In [15]:
#Init Trig Hierarchy
init_master_trig()
init_export_trigs()
test_exports()

# Run until here for setup

## Enable / disable GPS (1 = enabled, 0 = disabled) 

In [16]:
using_gps = 1 #1 is enabled, 0 is disabled

## Run next cell for transmit and "receive"

In [ ]:
import time
import threading

def track_user_input(stop_event):
    while not stop_event.is_set():
        user_input = input()
        if user_input.strip().lower() == 'stop':
            stop_event.set()

def live_elapsed_time():
    start_time = time.time()
    stop_event = threading.Event()

    input_thread = threading.Thread(target=track_user_input, args=(stop_event,))
    input_thread.daemon = True
    input_thread.start()

    try:
        while not stop_event.is_set():
            elapsed_time = time.time() - start_time
            print(f"\rElapsed time: {elapsed_time:.2f} seconds (Type 'stop' to end) ", end="")
            time.sleep(0.1)
    except KeyboardInterrupt:
        pass    
    print("STOPPING TRANSMIT")
    stop_transmit()
    print("STOPPING RECV")
    end_burst(use_gps = using_gps) 
    print("\nStopped!")

def main():
    time.sleep(0.5)
    print("INITIALIZING RECIEVE")
    init_recv()
    print("INIT RECV COMPLETE")
    while True:
        user_input = input("Type 'run' to start the collection, or 'exit' to quit: ").strip().lower()
        #RUN DATA COLLECTION
        if user_input == 'run':
            print("STARTING TRANSMIT")
            start_transmit()
            stop_transmit()
            start_transmit()
            print("TRANSMIT STARTED")
            print("STARTING RECV")
            start_burst(use_gps = using_gps)
            print("RECV STARTED")
            live_elapsed_time()
        elif user_input == 'exit':
            break
        else:
            print("Invalid input. Please type 'run' to start or 'exit' to quit.")

if __name__ == "__main__":
    main()


INITIALIZING RECIEVE
INIT RECV COMPLETE


Type 'run' to start the collection, or 'exit' to quit:  run


STARTING TRANSMIT
Sending data to DMA
Sending data to DMA
TRANSMIT STARTED
STARTING RECV
RECV STARTED
Elapsed time: 212.58 seconds (Type 'stop' to end) 

In [ ]:
# #Start Chirping
# start_transmit()
# stop_transmitru()
# start_transmit()

In [19]:
# #End Chirping
# stop_transmit()


In [20]:
# #Receive for 5 seconds(make sure tcpdump is on
# init_recv()
# start_burst(use_gps = 0)
# time.sleep(40)
# end_burst(use_gps = 0) 

In [ ]:
ol.cmac.getStats()

{'tx': {'packets': 178224250,
  'good_packets': 178224250,
  'bytes': 605249553000,
  'good_bytes': 605249553000,
  'packets_large': 0,
  'packets_small': 0,
  'bad_fcs': 0,
  'pause': 0,
  'user_pause': 0},
 'rx': {'packets': 178224250,
  'good_packets': 0,
  'bytes': 605249553000,
  'good_bytes': 0,
  'packets_large': 0,
  'packets_small': 0,
  'packets_undersize': 0,
  'packets_fragmented': 0,
  'packets_oversize': 0,
  'packets_toolong': 0,
  'packets_jabber': 0,
  'bad_fcs': 0,
  'packets_bad_fcs': 0,
  'stomped_fcs': 0,
  'pause': 0,
  'user_pause': 0},
 'cycle_count': 1561356838699}

In [22]:
# end_burst(use_gps = 0) 

In [23]:
# def generate_chirp(num_samples, f0, f1, sample_rate):
#     """
#     Generates a chirp signal.
    
#     Parameters:
#     num_samples (int): Number of samples in the chirp signal.
#     f0 (float): Starting frequency of the chirp in Hz.
#     f1 (float): Ending frequency of the chirp in Hz.
#     sample_rate (int): Sampling rate in samples per second.
    
#     Returns:
#     numpy.ndarray: The generated chirp signal as a numpy array of type int16.
#     """
#     t = np.linspace(0, num_samples / sample_rate, num_samples)
#     k = (f1 - f0) / (num_samples / sample_rate)
#     chirp_signal = np.cos(2 * np.pi * (f0 * t + 0.5 * k * t**2))
    
#     # Normalize to the range of int16
#     chirp_signal = np.int16(chirp_signal / np.max(np.abs(chirp_signal)) * (2**14 - 1)) # CHANGE BACK TO 14
    
#     return chirp_signal

# # Parameters
# # f0 = 100e6
# # f1 = 180e6
# # f0 = 220e6
# # f1 = 300e6
# f0 = 100e6
# f1 = 180e6
# sample_rate = 153.6e6 * 16# 156.3 MHz
# # signal_length = 64*50*16 # ~5 micro
# # signal_length = 64*50*16 * 2 # ~10 micro
# # signal_length = 64*64*16 * 2 # Max width
# # signal_length = 64*48*16*2
# signal_length = int(64*50*16 * 2  * 2)
# # signal_length = 64*50*16 * 2  * 2# ~20 micro
# # signal_length_20micro = 64*50*16 * 2  * 2# ~20 micro

# n_samples = signal_length//4  # Number of samples


# chirp_signal = generate_chirp(n_samples, f0, f1, sample_rate)

# # chirp_signal = np.concatenate([chirp_signal, np.zeros((signal_length_20micro-signal_length)//4, np.int16)])

# # f0 = 220e6
# # f1 = 300e6
# # chirp_signal += generate_chirp(n_samples, f0, f1, sample_rate)

# # f0 = 100e6
# # f1 = 180e6
# # chirp_signal += generate_chirp(n_samples, f0, f1, sample_rate)

# # chirp_signal = chirp_signal[::-1]